<a href="https://colab.research.google.com/github/nahyunryou/Machine_Learning/blob/main/2_3_making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [ ]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

In [ ]:
response.json()

{'type': 'FeatureCollection',
 'metadata': {'generated': 1665672195000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-09-12&endtime=2022-10-12',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.13.6',
  'count': 10355},
 'features': [{'type': 'Feature',
   'properties': {'mag': 1.98000002,
    'place': None,
    'time': 1665531896580,
    'updated': 1665532085440,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv73175062',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv73175062&format=geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'automatic',
    'tsunami': 0,
    'sig': 60,
    'net': 'hv',
    'code': '73175062',
    'ids': ',hv73175062,',
    'sources': ',hv,',
    'types': ',origin,phase-data,',
    'nst': 34,
    'dmin': None,
    'rms': 0.129999995,
    'gap': 182,
    'magType': 'md',
    'type': 'earthquake',
    'title

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [ ]:
earthquake_json = response.json() 
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [ ]:
earthquake_json['metadata']

{'generated': 1604267813000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2020-10-01&endtime=2020-10-31',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.10.3',
 'count': 13706}

Each element in the JSON array `features` is a row of data for our dataframe.

In [ ]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [ ]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1,
  'place': '50 km ENE of Susitna North, Alaska',
  'time': 1604102395919,
  'updated': 1604103325550,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak020dz5f85a',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak020dz5f85a&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 15,
  'net': 'ak',
  'code': '020dz5f85a',
  'ids': ',ak020dz5f85a,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 1.36,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.0 - 50 km ENE of Susitna North, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-148.9807, 62.3533, 5]},
 'id': 'ak020dz5f85a'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.98,None,1665531896580,1665532085440,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73175062,",",hv,",",origin,phase-data,",34.0,NaN,0.13,182.0,md,earthquake,M 2.0 -
1,1.30,"34 km ENE of Livengood, Alaska",1665531613003,1665531752201,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022d22k4t0,",",ak,",",origin,phase-data,",NaN,NaN,0.99,NaN,ml,earthquake,"M 1.3 - 34 km ENE of Livengood, Alaska"
2,2.80,"62 km NE of Teller, Alaska",1665531383090,1665534198040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us6000isvf,",",us,",",origin,phase-data,",9.0,0.283,1.04,110.0,ml,earthquake,"M 2.8 - 62 km NE of Teller, Alaska"
3,0.41,"85 km W of Adak, Alaska",1665531358310,1665598427640,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91693751,",",av,",",origin,phase-data,",4.0,NaN,0.10,212.0,ml,earthquake,"M 0.4 - 85 km W of Adak, Alaska"
4,-1.01,"30 km ESE of Port Heiden, Alaska",1665531048100,1665599925250,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91074368,",",av,",",origin,phase-data,",4.0,NaN,0.10,170.0,ml,earthquake,"M -1.0 - 30 km ESE of Port Heiden, Alaska"


### (Optional) Write Data to CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>